# Earthquake seismology with DAS

#### Todo List:
1. Reference earthquakes by Evid rather than dataframe index
2. Make a map of all of the detected events
3. Try some xcorr
4. Find something for JM to do

In [1]:
import h5py
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt
import numpy as np
import datetime
import pandas as pd
from dasquakes import sintela_to_datetime, open_sintela_file, get_file_number, local_earthquake_quicklook
import obspy
from obspy import UTCDateTime
from obspy.clients.fdsn import Client
from libcomcat.search import search
from libcomcat.dataframes import get_summary_data_frame

### Get a list of events from USGS

In [53]:
events = search(starttime = datetime.datetime(2022, 3, 17, 0, 0), 
                endtime   = datetime.datetime.now(),
                maxradiuskm=1000, 
                latitude=47.60, 
                longitude=-122.33)

print("Returned %s events" % len(events))

Returned 8270 events


In [54]:
event_df = get_summary_data_frame(events)
event_df = event_df.sort_values(by=['magnitude'],ascending=False)
event_df.iloc[0:5]

,id,time,location,latitude,longitude,depth,magnitude,alert,url,eventtype,significance
6026,us7000hhik,2022-06-15 11:56:33.191,off the coast of Oregon,44.5381,-130.1124,10.00,5.6,green,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,487
2848,us6000hfx7,2022-04-23 21:22:47.948,"185 km W of Port McNeill, Canada",50.4725,-129.6895,10.00,5.3,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,435
768,us7000gxl1,2022-03-27 01:12:22.059,off the coast of Oregon,44.1229,-129.4385,10.00,5.1,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,404
5459,us7000hf29,2022-06-04 23:59:10.007,"Haida Gwaii Region, Canada",51.6137,-131.4027,6.61,5.1,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,401
8249,us7000hth5,2022-07-26 05:46:22.472,"297 km S of Prince Rupert, Canada",51.6928,-131.1816,10.00,4.8,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,355


### Choose an event to analyze

In [55]:
detected_quakes = ['uw61835766','uw61832097','uw61836427',
                   'uw61846212','uw61819867','uw61850651','uw61855012']
# detected_quakes_whidbey = ['uw61832097','uw61819867','uw61855012','us7000gxl1']

# this_event_index = 0
# this_id=detected_quakes[this_event_index]

this_id = 'us7000hth5'

this_event = event_df[event_df.id==this_id]
t0 = this_event['time'].iloc[0]

this_event

,id,time,location,latitude,longitude,depth,magnitude,alert,url,eventtype,significance
8249,us7000hth5,2022-07-26 05:46:22.472,"297 km S of Prince Rupert, Canada",51.6928,-131.1816,10.0,4.8,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,355


### Load the DAS Data

In [63]:
# cable = 'seadasn'
cable = 'whidbey'
record_length = 10 #minutes

if cable == 'seadasn':
    prefix = 'seadasn'
    network_name = 'SeaDAS-N'
    if t0 < datetime.datetime(2022, 6, 20, 0, 0, 0):
        datastore='/data/data0/seadasn_2022-03-17_2022-06-20/'
    else:
        datastore='/data/data7/seadasn/'
        
elif cable == 'whidbey':
    prefix = 'whidbey'
    network_name='Whidbey-DAS'
    datastore = '/data/data5/Converted/'

data,dates,attrs = open_sintela_file(prefix,
                                     t0,
                                     datastore,
                                     number_of_files=record_length,
                                     verbose=True)
x_max=data.shape[1] * attrs['SpatialSamplingInterval']

/data/data5/Converted/whidbey_2022-07-26_05-46*.h5
/data/data5/Converted/whidbey_2022-07-26_05-47*.h5
/data/data5/Converted/whidbey_2022-07-26_05-48*.h5
/data/data5/Converted/whidbey_2022-07-26_05-49*.h5
/data/data5/Converted/whidbey_2022-07-26_05-50*.h5
/data/data5/Converted/whidbey_2022-07-26_05-51*.h5
/data/data5/Converted/whidbey_2022-07-26_05-52*.h5
/data/data5/Converted/whidbey_2022-07-26_05-53*.h5
/data/data5/Converted/whidbey_2022-07-26_05-54*.h5
/data/data5/Converted/whidbey_2022-07-26_05-55*.h5


In [64]:
low_cut = 2
hi_cut = 10

b,a = butter(2,(low_cut,hi_cut),'bp',fs=attrs['MaximumFrequency']*2)
data_filt = filtfilt(b,a,data,axis=0)

In [65]:
client = Client("IRIS")

t1 = UTCDateTime( t0.strftime("%Y-%m-%dT%H:%M:%S") )
dt = datetime.timedelta(minutes=record_length)
t2 = UTCDateTime( (t0+dt).strftime("%Y-%m-%dT%H:%M:%S") )

st = client.get_waveforms("UW", "NOWS", "--", "HNN", t1, t2, attach_response=True)
# st.decimate(5, strict_length=False, no_filter=False) #everything should be at 40Hz now
st2 = client.get_waveforms("IU", "COR", "00", "HH1", t1, t2, attach_response=True)

pre_filt = (low_cut/2, low_cut, hi_cut, hi_cut*2)
st.remove_response(output='VEL', pre_filt=pre_filt)
st2.remove_response(output='VEL', pre_filt=pre_filt)

1 Trace(s) in Stream:
IU.COR.00.HH1 | 2022-07-26T05:46:22.008393Z - 2022-07-26T05:56:21.998393Z | 100.0 Hz, 60000 samples

In [ ]:
# nombre = f"figures/{event_df.iloc[catalog_index]['Evid']}.png"
# local_earthquake_quicklook(dates,data_filt,st,st2,x_max,event_df,catalog_index,filename=nombre)


stitle=f"M {this_event['magnitude'].to_numpy()[0]}, "\
       f"{this_event['location'].to_numpy()[0]}, "\
       f"{str(this_event['time'].to_numpy()[0])[0:-10]} UTC"


local_earthquake_quicklook(dates,data_filt,st,st2,x_max,
                           stitle,das_vmax=0.02,
                           network_name=network_name)

In [62]:
this_event.url.to_list()[0]

'https://earthquake.usgs.gov/earthquakes/eventpage/us7000hth5'

In [ ]:
# from tqdm import tqdm
# xc = np.zeros((2*18000-1,174))

# for i in tqdm(range(174)):
#     xc[:,i] = np.correlate( data_filt[:,30]/max(data_filt[:,30]),
#                             data_filt[:,i*10]/max(data_filt[:,i*10]),
#                             mode='full')

In [ ]:
from tqdm import tqdm
xc = np.zeros((2*18000-1,174))

for i in tqdm(range(174)):
    xc[:,i] = np.correlate(np.sign(data_filt[:,30]),
                           np.sign(data_filt[:,i*10]),
                           mode='full')

In [ ]:
fig,ax=plt.subplots()
plt.imshow(xc.T,aspect='auto',vmin=-1000,vmax=1000)
# plt.plot(data_filt[:,30])
# plt.plot(data_filt[:,40])
plt.xlim(18000-1e3,18000+1e3)
plt.show()

In [ ]:
fig,ax=plt.subplots()
# plt.imshow(xc.T,aspect='auto',vmin=-0.1,vmax=0.1)
plt.plot(data_filt[:,40])
plt.plot(data_filt[:,30],alpha=0.5)
plt.xlim(6000,8000)
plt.show()